# Árboles de decisión

Son modelos predictivos, que gracias a su estructura de niveles y nodos, se pueden separar los datos, de tal manera, que se compara si tu dato en cuestión sobrepasa el umbral establecido en una característica específica. 

## Regresión

Primero el algoritmo utiliza todas las características y umbrales de división posibles, y con ello, seleccionar los umbrales que logren la mayor reducción de varianza.

$$
\text{Reducción de varianza} = \text{Varianza total} - \left( \frac{N_1}{N} \times \text{Var}(G1) + \frac{N_2}{N} \times \text{Var}(G2) \right)
$$

Esta formula compara la varianza total de las ramas con la varianza despues de ser ponderada por la cantidad de datos en cada rama.

## Clasificación

Igual que el de regresión, el algoritmo utiliza todas las características y umbrales de división posibles, y con ello, seleccionar que los umbrales que, ahora en este caso, logren la mayor reducción de impureza, calculada por medio de Gini o la Entropía.

$$
Gini = 1 - \sum p_i^2
$$

$$
Entropía = -\sum p_i \log_2 p_i
$$

$$
\text{Ganancia de Impureza} = \text{Impureza Nodo Padre} - \left( \frac{N_1}{N} \times \text{Impureza}(G1) + \frac{N_2}{N} \times \text{Impureza}(G2) \right)
$$

Esta formula compara la impureza del nodo padre con la impureza ponderada por la cantidad de datos en cada rama.

## Ventajas

+ Fácilmente interpretables gracias a las reglas y umbrales establecidos en el árbol de decisión. 
+ No es necesario escalar los datos

## Desventajas

+ Estos modelos pueden tender al overfitting debido a que se puede llegar a crear un árbol tan complejo, que crezca tanto que memorice los datos.
+ No tan buen poder predictivo.

----

# Random Forest

Utilizando el método de bootstrap, se realiza un muestreo aleatorio para crear varios árboles diferentes. Estos árboles son entrenados y para realizar las predicciones cambiará dependiendo de si es regresión o clasificación. 

+ Si es regresión se promediará la predicción de todos los árboles creados.
+ Si es de clasificación, se toma la categoría más repetida.

## Ventajas

+ Mayor poder predictivo de comparado a los arboles de decisión normales.
+ Gracias al muestreo aleatorio se reduce en gran medida el overfitting.
+ No es necesario escalamiento de datos.

## Desventajas

+ Pierde interpretabilidad.
+ Es mas costoso computacionalmente.

----

# Prueba de poder

Es una técnica utilizada para determinar el tamaño de muestra necesario para lograr observar un efecto con una probabilidad determinada, así como tambie, para ver que tan probable es que mi estudio detecte un efecto real (si es que existe).

## Error tipo I $\alpha$

Falso positivo: rechazamos $H_0$ cuando en realidad es verdadera

## Error tipo II ($\beta$)

Falso negativo: No rechazamos $H_0$ cuando en realidad $H_A$ es la verdadera

| Tipo de Error | Decisión Tomada  | Realidad         | Consecuencia |
|--------------|-----------------|-----------------|--------------|
| **Tipo I $\alpha$** | Cambiamos a rojo | No había diferencia | Cambio innecesario, pérdida de recursos |
| **Tipo II $\beta$** | No cambiamos a rojo | Rojo era mejor | Perdemos una oportunidad de mejora |

## Elementos clave
- Potencia : (1 - $\beta$) Es la probabilidad de rechazar $H_0$ cuando $H_A$ es negativa (Error tipo II). Usualmente queremos tener $\beta$ en 80%
- Nivel de significancia: $\alpha$ Es la probabilidad de rechazar $H_0$ cuando $H_0$ es verdadera (error tipo I)

## Tamaño de Muestra


$$
N = \frac{(Z_{\alpha/2} + Z_{\beta})^2 \cdot 2p(1-p)}{\Delta^2}
$$

donde:
- **$Z_{\alpha/2}$** es el umbral para rechazar $H_0$ (ej. 1.96 para $\alpha = 0.05$.
- **$Z_{\beta}$** se elige según la potencia deseada (ej. 0.84 para 80% de potencia).
- **$p(1-p)$** es la varianza de la tasa de conversión.
- **$\Delta$** es la diferencia esperada entre A y B ($p_B - p_A$).

## Con StatsModels

```python
PA = 0.1
PB = 0.12

N = sms.proportion_effectsize(PA, PB)  # Efecto
sample_size = sms.NormalIndPower().solve_power(effect_size=N, alpha=0.05, power=0.8, ratio=1)
sample_size
```

----

# Gradient Boosting

Es un modelo en el que se van construyendo árboles de decisión de manera iterativa, donde el modelo aprende con los residuales del árbol anterior, y con ello, se actualiza haciendo un modelo para equivocarse menos. Al igual que en el descenso en gradiente, para actualizar el los árboles con cada iteración se utiliza el factor $\alpha$ de learning rate y se repite hasta converger.

La predicción del modelo sería de la siguiente manera:

$$F_m(x) = F_{m - 1}(x) + \nu (\text{nuevo árbol})$$

La predicción del árbol anterior será el promedio de los residuales de cada hoja.

En clasificación, la predicción estaría en términos de log-odds.

---

# XGBoost

Este modelo es la versión optimizada del Gradient Boosting. En este es un modelo aditivo de árboles, en otras palabras, el modelo final se construye sumando árboles, de la siguiente manera:

$$
\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)
$$

Donde:
- $\hat{y}_i^{(t)}$ es la predicción del ejemplo $i$ en la iteración $t$
- $f_t(x_i)$ es el nuevo árbol que se entrena en la iteración $t$

Con cada iteración, se busca que el nuervo árbol $f_t$ minimice la contribución marginal a la función de pérdida total:

$$
\mathcal{L}^{(t)} =
\underbrace{
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)}) + \sum_{k=1}^{t-1} \Omega(f_k)
}_{\text{Parte ya construida (constante en esta iteración)}} +
\underbrace{
\sum_{i=1}^n \left[ l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) - l(y_i, \hat{y}_i^{(t-1)}) \right] + \Omega(f_t)
}_{\text{Lo que añade el nuevo árbol $f_t$}}
$$

Donde $\Omega(f) = \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T} w_j^2$ es una penalización por la complejidad del árbol $f_k$

Dado que esta función de pérdada es muy compleja, se utiliza la expansión de Taylor para aproximar esta función por medio de polinomios, y de esta manera hacerla más facil de optimizar:

$$
f(x) \approx f(a) + f'(a)(x - a) + \frac{1}{2} f''(a)(x - a)^2
$$

Despues de áplicar la expansión de Taylor en XGBoost y utilizando el gradiente y hessiano, se llega, a que la función que se optimiza en cada iteración es:


$$
\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)
$$

Cada nuevo árbol $f_t$ asigna un valor constante $w_j$, entonces, sustituyendo esto en la antigua fución objetivo y escribiendo $\Omega(f)$ explícitamente:

$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
\sum_{i \in R_j} g_i w_j + \frac{1}{2} \sum_{i \in R_j} h_i w_j^2
\right] + \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T} w_j^2
$$

Simplificando:


$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
\right] + \gamma T
$$

Donde:

$G_j = \sum_{i \in R_j} g_i$

$H_j = \sum_{i \in R_j} h_i$

Luego de minimizar el término por hoja, derivando con respecto a $w_j$, llegamos al siguiente output value:

$$
\textbf{Output value} = w_j^* = -\frac{G_j}{H_j + \lambda}
$$

Luego se calcula el similarity score para evaluar cuanto mejora la pérdida por haber utilizado el óptimo:

$$
\text{Similarity Score} = \frac{G_j^2}{H_j + \lambda}
$$

Gracias al similarity score ahora podemos calcular el Gain, y esta medida nos ayudará a ver si es conveniente realizar un split, es decir, aumentar el grado de complejidad de nuestro árbol:

$$
\text{Gain} = \frac{1}{2} \left( \text{Similarity}_\text{izq} + \text{Similarity}_\text{der} - \text{Similarity}_\text{padre} \right) - \gamma
$$

Luego de realizar todo lo anterior, podremos realizar la predicción final, que consiste en realizar la suma de todos los árboles si es regresión o aplicar la función sigmoide si es clasificación:

$$
\hat{y}_i = F_0(x_i) + \sum_{t=1}^{M} f_t(x_i)
$$

$$
p_i = \frac{1}{1 + e^{-\hat{y}_i}}
$$

## Resumen

| Concepto            | Fórmula                                                       | Interpretación                                |
|---------------------|----------------------------------------------------------------|------------------------------------------------|
| Output value        | $w_j = -\frac{G_j}{H_j + \lambda}$                         | Valor óptimo que predice una hoja             |
| Similarity score    | $\frac{G_j^2}{H_j + \lambda}$                              | Calidad de una hoja                           |
| Gain (split)        | Ver fórmula anterior                                           | Reducción esperada en la pérdida si se divide |

----

# LightGBM

Es un modelo, donde a diferencia de XGBoost, es más ligero computacionalmente y más rápido, debido a que suele funcionar con bins y así como tambien, generar árboles que no sean simétricos, con mayor o menor profundidad en ciertas ramas.

Gracias a este crecimiento asimétrico, este modelo puede encontrar mejores cortes en los nodos, pero esto tambien puede llevar a que sea más propenso al overfitting.

A diferencia de XGBoost, que construye árboles nivel por nivel (level-wise), LightGBM crece los árboles de forma hoja por hoja (leaf-wise)

----

# CatBoost

Tambien es un modelo de gradient boosting, sin embargo, este destaca por su grán capacidad de manejo de variables categóricas sin realizar encoding manual. Este modelo tambien genera árboles simétricos, lo que lo vuelve más robusto ante el overfitting.

La forma en la que CatBoost maneja las variables categóricas funciona de la siguiente manera:

1. Ordena las filas de forma aleatoria.
2. Para cada fila se codifica la categoría con el promedio del target acumulato hasta ese momento con las observaciones anteriores.

Dado que nunca se incluye el target de la fila actual, sino de la enteror, esto no introduce data leakege en el modelo.

Esto es realizado con varias permutaciones del dataset, para al final promediar los resultados. Con esto se logra un encoding estable.

----

# Comparativa de modelos de gradient boosting

| Característica              | **XGBoost**                                                | **LightGBM**                                               | **CatBoost**                                                  |
|-----------------------------|------------------------------------------------------------|-------------------------------------------------------------|----------------------------------------------------------------|
| **Velocidad**               | Rápido, pero más lento que LightGBM y CatBoost             | 🔥 Muy rápido gracias a histogramas y leaf-wise growth      | Rápido, aunque un poco más lento que LightGBM                  |
| **Precisión**               | Alta                                                       | Alta, a veces mejor con buen tuning                         | Muy alta, especialmente con categóricas                        |
| **Variables categóricas**   | ❌ No las maneja (requiere encoding manual)                | ❌ No las maneja (requiere encoding manual)                 | ✅ Soporte nativo + regularización secuencial                  |
| **Uso de memoria**          | Moderado                                                   | ✅ Muy eficiente (binning)                                   | Similar a XGBoost                                              |
| **Manejo de missing values**| ✅ Automático                                               | ✅ Automático                                                | ✅ Automático                                                   |
| **Soporte GPU**             | ✅ Sí (bastante estable)                                   | ✅ Sí (muy rápido)                                           | ✅ Sí (algo más limitado)                                      |
| **Instalación**             | Fácil (`pip install xgboost`)                             | Fácil (`pip install lightgbm`)                              | Un poco más pesada (`pip install catboost`)                   |
| **Documentación**           | Excelente                                                  | Buena                                                       | Muy buena                                                     |
| **Interacción con sklearn** | Muy buena                                                  | Muy buena                                                   | Muy buena                                                     |
| **Tolerancia al orden**     | ✅ Neutral                                                  | ✅ Neutral                                                   | ⚠️ Sensible (por codificación secuencial)                      |

## Situaciones de cada modelo

| Situación                                                  | Recomendación                                      |
|------------------------------------------------------------|----------------------------------------------------|
| Dataset tabular pequeño o mediano                          | ✅ XGBoost o CatBoost                               |
| Dataset grande, muchas variables numéricas                 | ✅ LightGBM                                         |
| Muchas variables categóricas sin preprocesamiento          | ✅ CatBoost (manejo nativo y robusto)              |
| Quieres algo robusto y estable con buen soporte            | ✅ XGBoost (muy probado en producción y Kaggle)     |
| Entrenamiento rápido con buen desempeño                    | ✅ LightGBM                                         |
| Quieres interpretabilidad con SHAP                         | ✅ Cualquiera, pero CatBoost da mejores resultados con categóricas |
| Necesitas buen rendimiento sin mucho tuning                | ✅ CatBoost (buenos defaults)                       |
| Ya tienes pipeline con OneHot/Target Encoding              | ✅ XGBoost o LightGBM                               |
| Tuning automático (Optuna, GridSearchCV, etc.)             | ✅ LightGBM (rápido y convergente)                  |
| Producción en sistemas legacy o APIs bien documentadas     | ✅ XGBoost (mayor madurez, más integración)         |
| Clasificación multi-label o problemas no estándar          | ✅ XGBoost (soporte más flexible)                   |


# Partial dependece plots

Dejas una variable fija y sacas el promedio de la predicción y sirve para ver como afecta esa variable a la predicción (relación de las variables)

----

# Feature Importance

## Weight

Veces que se usa una variable en el modelo

## Gain

En promedio cuanto baja la función de pérdida una variable

----

#